In [1]:
!pip install git+git://github.com/seemee/exactcover.git#egg=exactcover

  Cloning git://github.com/seemee/exactcover.git to /tmp/pip-install-xbqd1s3s/exactcover
  Running command git clone -q git://github.com/seemee/exactcover.git /tmp/pip-install-xbqd1s3s/exactcover
  Created wheel for exactcover: filename=ExactCover-0.1-cp36-cp36m-linux_x86_64.whl size=20795 sha256=b77de2b71edba844538d37ca29e9834fb8c04af2f054c0665ba2e533a2bc5e50
  Stored in directory: /tmp/pip-ephem-wheel-cache-p03yb4w3/wheels/d9/1a/40/50a35a078ec5cbb53d17f48a4d94ce402a94697350fb348ce5
Successfully built exactcover


In [13]:
import exactcover
import pprint

# The twelve pentominos, along with their names.
pentominos = {
    'f': [(1, 0), (2, 0), (0, 1), (1, 1), (1, 2)],
    'i': [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)],
    'l': [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)],
    'n': [(1, 0), (1, 1), (0, 2), (1, 2), (0, 3)],
    'p': [(0, 0), (1, 0), (0, 1), (1, 1), (0, 2)],
    't': [(0, 0), (1, 0), (2, 0), (1, 1), (1, 2)],
    'u': [(0, 0), (2, 0), (0, 1), (1, 1), (2, 1)],
    'v': [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)],
    'w': [(0, 0), (0, 1), (1, 1), (1, 2), (2, 2)],
    'x': [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],
    'y': [(1, 0), (0, 1), (1, 1), (1, 2), (1, 3)],
    'z': [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)],
}

def rotations(shape):
    """List the rotations and reflections of a pentomino."""
    # The 8 orientations are combinations of 3 primitive operations, x-flip,
    # y-flip, and transpose.
    def xflip(shape):
        return [(4 - x, y) for x, y in shape]
    def yflip(shape):
        return [(x, 4 - y) for x, y in shape]
    def transpose(shape):
        return [(y, x) for x, y in shape]
    def align(shape):
        mx = min(x for x, y in shape)
        my = min(y for x, y in shape)
        return sorted((x - mx, y - my) for x, y in shape)

    # Helper to add unique shapes to a list
    out = []
    def add(shape):
        rotation = align(shape)
        if rotation not in out:
            out.append(rotation)

    add(shape)
    add(transpose(xflip(shape)))
    add(xflip(yflip(shape)))
    add(transpose(yflip(shape)))
    add(xflip(shape))
    add(yflip(shape))
    add(transpose(shape))
    add(transpose(xflip(yflip(shape))))
    return out


def positions(shape, width, height, world):
    """List the positions a pentomino can be in.
    'world' is a set containing all of the legal squares on the board.
    'width' and 'height' are the maximum extents of world.
    """
    for y in range(height):
        for x in range(width):
            new_shape = [(x + xx, y + yy) for xx, yy in shape]
            if set(new_shape).issubset(world):
                yield new_shape


def board():
    """List the legal squares on the board.
    The board is a standard 8x8 chess board with the center 4 squares removed.
    """
    b = set((x, y) for x in range(8) for y in range(8)
            if not (3 <= x < 5 and 3 <= y < 5))
    return b


def matrix():
    """Compute the set of covers representing the problem."""
    b = board()

    covers = []
    for name, shape in pentominos.items():
        for rotation in rotations(shape):
            for position in positions(rotation, 8, 8, b):
                covers.append([name] + sorted(position))
    return covers


def solution_str(solution):
    """Turn a covering into a string picture representation."""
    grid = [[' ' for i in range(8)] for j in range(8)]

    # Mark unoccupied squares.
    for x, y in board():
        grid[y][x] = '.'

    # Mark each pentomino by its name.
    for row in solution:
        c = row[0]
        for x, y in row[1:]:
            grid[y][x] = c

    return "\n".join(''.join(row) for row in grid)


def main():
    m = matrix()

    print("Example covering:")
    # Take the first result from the iterator.
    solution = next(exactcover.Coverings(m))
    pprint.pprint(solution)
    print()
    print(solution_str(solution))
    print()

    # Count the number of results returned by the iterator.
    print('There are',len(list(exactcover.Coverings(m))),'unique tilings.')

if __name__ == '__main__':
    main()

Example covering:
(['x', (1, 1), (2, 0), (2, 1), (2, 2), (3, 1)],
 ['l', (0, 0), (0, 1), (0, 2), (0, 3), (1, 0)],
 ['f', (0, 4), (1, 2), (1, 3), (1, 4), (2, 3)],
 ['u', (3, 0), (3, 2), (4, 0), (4, 1), (4, 2)],
 ['v', (5, 0), (5, 1), (5, 2), (6, 0), (7, 0)],
 ['z', (5, 3), (6, 1), (6, 2), (6, 3), (7, 1)],
 ['n', (6, 4), (6, 5), (7, 2), (7, 3), (7, 4)],
 ['t', (5, 6), (6, 6), (7, 5), (7, 6), (7, 7)],
 ['w', (3, 6), (4, 5), (4, 6), (5, 4), (5, 5)],
 ['i', (2, 7), (3, 7), (4, 7), (5, 7), (6, 7)],
 ['y', (0, 5), (1, 5), (2, 4), (2, 5), (3, 5)],
 ['p', (0, 6), (0, 7), (1, 6), (1, 7), (2, 6)])

llxuuvvv
lxxxuvzz
lfxuuvzn
lff  zzn
ffy  wnn
yyyywwnt
pppwwttt
ppiiiiit

There are 520 unique tilings.


In [15]:
m=matrix()
%timeit for _ in exactcover.Coverings(m): pass

1 loop, best of 3: 809 ms per loop


In [17]:
import exactcover


def print_solution(n, solution):
    a = [
        [ "." for col in range(n) ]
        for row in range(n)
    ]
    for entry in solution:
        row = int(entry[0][1:])
        col = int(entry[1][1:])
        a[row][col] = "*"

    for row in a:
        print("".join(row))

n = 8
matrix = []
for row in range(n):
    for col in range(n):
        matrix.append((
            "r%d" % row,
            "c%d" % col,
            "d%d" % (row + col),
            "a%d" % (row + n-1 - col)
        ))

# The diagonal and antidiagonal constraints are secondary,
# since not every diagonal will have a queen on it.
secondary = set(( "d%d" % i for i in range(2*n-1) ))
secondary.update(set(( "a%d" % i for i in range(2*n-1) )))
count=0
for solution in exactcover.Coverings(matrix, secondary):
    count+=1
    print(str(count)+':')
    print_solution(n, solution)
print('Total solutions',count)


1:
*.......
......*.
....*...
.......*
.*......
...*....
.....*..
..*.....
2:
*.......
......*.
...*....
.....*..
.......*
.*......
....*...
..*.....
3:
*.......
.....*..
.......*
..*.....
......*.
...*....
.*......
....*...
4:
*.......
....*...
.......*
.....*..
..*.....
......*.
.*......
...*....
5:
.*......
...*....
.....*..
.......*
..*.....
*.......
......*.
....*...
6:
.*......
....*...
......*.
...*....
*.......
.......*
.....*..
..*.....
7:
.*......
....*...
......*.
*.......
..*.....
.......*
.....*..
...*....
8:
.*......
.....*..
*.......
......*.
...*....
.......*
..*.....
....*...
9:
.*......
.....*..
.......*
..*.....
*.......
...*....
......*.
....*...
10:
.*......
......*.
..*.....
.....*..
.......*
....*...
*.......
...*....
11:
.*......
......*.
....*...
.......*
*.......
...*....
.....*..
..*.....
12:
.*......
.......*
.....*..
*.......
..*.....
....*...
......*.
...*....
13:
..*.....
*.......
......*.
....*...
.......*
.*......
...*....
.....*..
14:
..*.....
....*...

In [18]:
%timeit for _ in exactcover.Coverings(matrix, secondary): pass

1000 loops, best of 3: 417 µs per loop
